<a href="https://colab.research.google.com/github/David9857/VQA/blob/main/train/KVQA1_open_ended.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/files/data_path.zip /content
!unzip data_path.zip "data/jsons/*" "data/pic/*" "data/ques_embeddings/bioelmo/*" "data/answer_word_frequency.csv" "data/answer_word_frequency.xlsx" -d /content

In [ ]:
# %%bash
# cd /content/drive/MyDrive/data
# zip -r knowledge_embs.zip knowledge_embeddings/

In [ ]:
!cp /content/drive/MyDrive/data/knowledge_embs.zip /content/data
!unzip /content/data/knowledge_embs.zip -d /content/data

In [ ]:
!ls /content/data/knowledge_embeddings | wc -l

32761


In [ ]:
# %cd /content
# !unzip data_path.zip "data/ques_embeddings/bioelmo/*" -d /content

In [ ]:
# %cd /content/drive/MyDrive/VQACode

In [4]:
%cd /content/drive/MyDrive/VQACode

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import pandas as pd
import time
import pathlib
from utils.load_data import DataLoader
from utils.evaluation import AnswerEvaluator
from utils.training_toolkit import CustomSchedule, loss_function
from models.Transformer.transformers import VQATransformer
from models.Transformer.masks import create_masks

/content/drive/MyDrive/VQACode


### Set up Arguments

In [5]:
num_layers=2
d_model=512
num_heads=8
dff=2048
maximum_position_encoding=10000
EPOCHS = 30
batch_size = 64
cnn_type = 'inception'
embedding = 'bioelmo'  # choose from ['w2v', 'bioelmo', 'biobert', 'bluebert', 'large_biobert', 'elmo']
data_augmentation = True


In [6]:
####### DO NOT CHANGE VALUES OF THIS BLOCK IF YOU ARE NOT THE DEVELOPER ##########

check_point_path = './check_point/transformer/open_ended/' + embedding +'/' + cnn_type + '_' + str(num_layers)
saving_folder = './open_ended_results/transformer/' + embedding + '/'
save_result_path = saving_folder + cnn_type + '_' + str(num_layers) + '.csv'

emb_size = 1024
pe_output = 36 + 1
MAX_LENGTH = pe_output
if cnn_type == 'inception':
    img_shape = [299, 299]
    img_padding = tf.TensorShape([299, 299, 3])
if cnn_type in ['resnet', 'resnet_v2', 'dense_net', 'vgg19']:
    img_shape = None
    img_padding = tf.TensorShape([224, 224, 3])

if embedding == 'bioelmo':
    pe_input = 38
elif embedding == 'elmo':
    pe_input = 42
elif embedding == 'biobert':
    pe_input = 72
    emb_size = 768
elif embedding == 'bluebert':
    pe_input = 69
elif embedding == 'large_biobert':
    pe_input = 60  
elif embedding == 'w2v':
    pe_input = 48
    emb_size = 200
elif embedding == 'bert':
    pe_input = 72
    emb_size = 1024
else:
    raise TypeError("Wrong embedding type")
    
if data_augmentation:
    aug = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.RandomFlip(),
                               tf.keras.layers.experimental.preprocessing.RandomRotation(0.05)])

#### 

### Create Datasets

In [7]:
# create train, val, test dataset
kn_input = 155

data_loader = DataLoader('/content/data', emb_folder=embedding)
full_dataset, tokenizer = data_loader.create_dataset('open_ended')
vocab_size=len(tokenizer.index_word) + 1
Data_SET_SIZE = len(full_dataset)
train_size = int(0.52 * Data_SET_SIZE)
val_size = int(0.30 * Data_SET_SIZE)
test_size = int(0.18 * Data_SET_SIZE)
train_set = full_dataset.take(train_size)
val_test_ds = full_dataset.skip(train_size)
val_set = val_test_ds.take(val_size)
test_ds = val_test_ds.skip(val_size)
test_set = test_ds.take(test_size)

batch_train_set = train_set.padded_batch(batch_size, padded_shapes=((img_padding, tf.TensorShape([pe_input, emb_size]), tf.TensorShape([kn_input, emb_size])),
                                            tf.TensorShape([pe_output-1]), []), drop_remainder=True)
batch_test_set = test_set.padded_batch(1, padded_shapes=((img_padding, tf.TensorShape([pe_input, emb_size]), tf.TensorShape([kn_input, emb_size])),
                                                         tf.TensorShape([pe_output-1]), []), drop_remainder=True)

11111


In [ ]:
for i in enumerate(batch_train_set):
  (batch, (img_question, tar, q_id)) = i
  print(batch,img_question[0].shape,img_question[1].shape,img_question[2].shape,tar.shape,q_id.shape)
  break

0 (64, 224, 224, 3) (64, 38, 1024) (64, 155, 1024) (64, 36) (64,)


In [ ]:
maxl = 0
for (img_question, tar, q_id) in full_dataset.as_numpy_iterator():
  # print(img_question[0].shape,img_question[1].shape,tar.shape,q_id)
  # print(img_question[2].shape)
  l = img_question[2].shape[0]
  if l > maxl:
    maxl = l
print(maxl)

In [ ]:
import numpy as np
# validate saved features and calculate max length of all questions
ques_id = 48
emb = np.load('/content/data/ques_embeddings/bioelmo/'+str(ques_id)+'.npy')
# length = emb.shape[0]
print(ques_id,'shape is', emb.shape)        


### 

### Define Models and Related Functions 

In [8]:
transformer = VQATransformer(num_layers, d_model, num_heads, dff, vocab_size, pe_input, pe_output,
                          pretrained_cnn_type=cnn_type)

learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, check_point_path, max_to_keep=5)

87916544/87910968 [==============================] - 1s 0us/step


In [9]:
@tf.function()
def train_step(img, question, kn, tar):
    if data_augmentation:
        img = aug(img)
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(question, tar_inp)
    with tf.GradientTape() as tape:
        predictions, _ = transformer(question, img, kn, tar_inp,
                                     True,
                                     enc_padding_mask,
                                     combined_mask,
                                     dec_padding_mask)
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(tar_real, predictions)

In [10]:
def evaluate(question, img, kn):
    end_token = tf.constant(tokenizer.texts_to_sequences(['<end>']), tf.int32)
    output = dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    for i in range(MAX_LENGTH):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
            question, output)
        predictions, attention_weights = transformer(question,
                                                     img,
                                                     kn,
                                                     output,
                                                     False,
                                                     enc_padding_mask,
                                                     combined_mask,
                                                     dec_padding_mask)

        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        if predicted_id == end_token:
            return tf.squeeze(output, axis=0), attention_weights
        output = tf.concat([output, predicted_id], axis=-1)
    return tf.squeeze(output, axis=0), attention_weights

### 

### Train the Model 

In [16]:
for epoch in range(EPOCHS):
    start = time.time()
    train_loss.reset_states()
    train_accuracy.reset_states()
    for (batch, (img_question, tar, _)) in enumerate(batch_train_set):
        train_step(img_question[0], img_question[1], img_question[2], tar)
        if batch % 50 == 0:
            print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
                epoch + 1, batch, train_loss.result(), train_accuracy.result()))
    # if (epoch + 1) % 2 == 0:
    #     ckpt_save_path = ckpt_manager.save()
    #     print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
    #                                                         ckpt_save_path))

    print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1,
                                                train_loss.result(),
                                                train_accuracy.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.0320 Accuracy 0.1009
Epoch 1 Batch 50 Loss 0.0229 Accuracy 0.0950
Epoch 1 Batch 100 Loss 0.0225 Accuracy 0.0959
Epoch 1 Loss 0.0230 Accuracy 0.0964
Time taken for 1 epoch: 114.05688762664795 secs

Epoch 2 Batch 0 Loss 0.0274 Accuracy 0.1004
Epoch 2 Batch 50 Loss 0.0230 Accuracy 0.0948
Epoch 2 Batch 100 Loss 0.0218 Accuracy 0.0959
Epoch 2 Loss 0.0225 Accuracy 0.0963
Time taken for 1 epoch: 114.7628288269043 secs

Epoch 3 Batch 0 Loss 0.0282 Accuracy 0.1004
Epoch 3 Batch 50 Loss 0.0232 Accuracy 0.0946
Epoch 3 Batch 100 Loss 0.0219 Accuracy 0.0958
Epoch 3 Loss 0.0222 Accuracy 0.0963
Time taken for 1 epoch: 113.8757996559143 secs

Epoch 4 Batch 0 Loss 0.0269 Accuracy 0.1000
Epoch 4 Batch 50 Loss 0.0223 Accuracy 0.0950
Epoch 4 Batch 100 Loss 0.0219 Accuracy 0.0959
Epoch 4 Loss 0.0221 Accuracy 0.0965
Time taken for 1 epoch: 114.18289566040039 secs

Epoch 5 Batch 0 Loss 0.0256 Accuracy 0.1013
Epoch 5 Batch 50 Loss 0.0207 Accuracy 0.0954
Epoch 5 Batch 100 Loss 0.0200 Acc

In [ ]:
ckpt_save_path = ckpt_manager.save()

### 

### Predicting and Evaluating 

In [17]:
true_answers_list = []
predicted_answers_list = []
ques_id_list = []
print('Start predicting...')
for (batch, (img_question, target, ques_id)) in enumerate(batch_test_set):
    target = target.numpy()
    target = target[0]
    true_answer = []
    for i in target:
        if i == 0:
            break
        else:
            true_answer.append(tokenizer.index_word[i])
    true_answer = " ".join(true_answer[1: -1])

    prediction, attention = evaluate(img_question[1], img_question[0], img_question[2])
    p = prediction.numpy()
    predict_answer = [tokenizer.index_word[i] for i in p][1:]
    predict_answer = " ".join(predict_answer)
    true_answers_list.append(true_answer)
    predicted_answers_list.append(predict_answer)
    ques_id_list.append(ques_id)
    print("predicted answer: " + str(batch), end='\r', flush=True)

Start predicting...


In [18]:
data = {"true answer": true_answers_list, "predicted answer": predicted_answers_list, "ques_id": ques_id_list}
df = pd.DataFrame(data)
if not pathlib.Path(saving_folder).exists():
    pathlib.Path(saving_folder).mkdir(parents=True, exist_ok=True)
name = save_result_path
df.to_csv(name)
print("complete writing", name)

complete writing ./open_ended_results/transformer/bioelmo/inception_2.csv


In [19]:
scores = AnswerEvaluator(name).evaluate()

Accuracy: 29.19
Exact Match: 22.91
F1 Score: 28.0
BLEU-1: 37.05
BLEU-2: 24.52
BLEU-3: 17.2
BLEU-4: 10.4
